In [1]:
import pandas as pd
import micom
from micom import load_pickle
from micom.media import minimal_medium
from micom.workflows import workflow

In [2]:
max_procs = 10
processes = []


In [3]:
def media_and_gcs(sam):
    com = load_pickle('./comunity_' + sam + ".pickle")
    print(com)
    # Get growth rates
    sol = com.cooperative_tradeoff(fraction=0.9)
    rates = sol.members["growth_rate"].copy()
    rates["community"] = sol.growth_rate
    rates.name = sam

    # Get the minimal medium
    med = minimal_medium(com, 0.95*sol.growth_rate, exports = True)
    med.name = sam

    #Get fluxes
    com.medium = med[med > 0]
    sol = com.cooperative_tradeoff(fraction = 0.9, pfba = True, fluxes = True)
    fluxes = sol.fluxes
    fluxes["sample"] = sam
    return {"medium": med, "gcs": rates, "fluxes":fluxes}

In [4]:
gcs = pd.DataFrame()
media = pd.DataFrame()
fluxes = pd.DataFrame()

In [ ]:
results = workflow(media_and_gcs, ['6', '19'] , max_procs)

In [ ]:
for s in results:
    gcs = gcs.append(s["gcs"])
    media = media.append(s["medium"])
    fluxes = fluxes.append(s["fluxes"])

gcs.to_csv("growth_rates.csv")
media.to_csv("minimal_media.csv")
fluxes.to_csv("minimal_fluxes.csv.gz", compression = "gzip")
